# About: Shibboleth認証の設定
---
MoodleにShibboleth認証の設定を追加します

## 概要

「021-Moodleコンテナの起動.ipynb」で構築したMoodle環境をShibbolethと連携して認証が行えるように設定します。

![処理の流れ](images/moodle-221-01.png)

設定手順の主な流れを以下に示します。

1. HTTPサーバ、Shibbleth SPで利用する証明書と秘密鍵を配置する
1. httpd, shibd の設定ファイルを配置する
1. Shibbolethコンテナの構成を記述した `docker-compose.yml` を配置する
1. 新たな構成でコンテナを起動しなおす
1. Moodleの管理画面でShibbolethプラグインの設定を行う

Shibbolethによる認証を行う場合は時刻同期を有効にすることを推奨します。VCノードで時刻同期を行うには「[Release/20.10.0 -- 2.機能追加](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=32677360#id-%E3%83%AA%E3%83%AA%E3%83%BC%E3%82%B9%E3%83%8E%E3%83%BC%E3%83%88-Release/20.10.0(2020/10/30))」に記されているようにVCコントローラへの設定を行う必要があります。設定を行う場合はVCノードを構築する前にOCS運用担当者に設定変更を依頼してください。

### コンテナ構成について

「021-Moodleコンテナの起動.ipynb」で構築した環境は次図のようなコンテナ構成となっています。

![変更前の構成](images/moodle-071-02.png)

 MoodleコンテナではMoodle をHTTP(80)で外部に公開する Apache HTTP server(httpd) と MoodleのPHPファイルを実行する php-fpm が実行されています。httpdとphp-fpmはUNIXドメインソケットを経由して通信を行っています。

MoodleとShibbolethが連携できるように次図のようにコンテナ構成を変更します。

![変更後の構成](images/moodle-221-02.png)

この構成ではHTTPS(443)での接続を行なうhttpdとShibboleth SPの処理を行なうshibdを実行するShibbolethコンテナをMoodleコンテナの前段に配置しています。

Shibbolethコンテナのhttpdで受け付けた外部からのリクエストはUNIXドメインソケットを経由してMoodleコンテナのphp-fpmで処理します。この連携を行うためにホスト環境にあるUNIXドメインソケットのファイルをShibbolethコンテナとMoodleコンテナからbindマウントして、それぞれのコンテナから同一のソケットが見えるように設定します。 この構成では元々Moodleコンテナで実行していたhttpdは不要なので停止させます。


「071-HTTPS接続に変更する.ipynb」を既に実行している状態でこのNotebookを実行する場合は、 Moodleコンテナの前段に配置されているhttpdコンテナをShibbolethコンテナに置き換えることになります。ShibbolethコンテナはHTTPS接続でMoodleコンテナを公開するのでhttpdコンテナの機能を包含しており置き換えることが可能です。

### Shibboleth SPの設定について

#### 学認フェデレーションに参加する場合

学認技術ガイドの「[SPセッティング　- 接続までに必要なセッティング](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158187)」に沿った設定を行います。

Shibboleth SPのインストールやshibboleth2.xmlのentityIDの設定、サーバ証明書の設定についてはShibbolethコンテナに組み込まれています。学認技術ガイドの「[shibboleth2.xmlファイル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158266)」に示されている以下の項目については参加するフェデレーションにより異なる指定を行う必要があります。

* 2. DSサーバの参照設定を行います。
* 3. メタデータの自動更新設定を行います。

これらの項目についての設定を「[3.2 学認フェデレーションに関するパラメータ](#学認フェデレーションに関するパラメータ)」「[5.2 メタデータを検証するための証明書の指定](#メタデータを検証するための証明書の指定)」にて行います。

> 設定手順は学認技術ガイドに記されている最低限のものとなります。個別の状況に対応するより詳細な設定が必要となる場合は構築後に`shibboleth2.xml`などの設定ファイルを編集してください。

#### 学認フェデレーションに参加せずに運用する場合

学認技術ガイドの「[学内システムとして構築する場合の設定](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158282)」に沿った設定を行います。


「学内システムとして構築する場合の設定」では、以下の設定を行う手順が示されています。

1. SPは、DSを経由せず、特定のIdPのみ信頼する
1. IdPは、特定のSPのみ信頼する

このNotebookでは SP に対する設定のみを実行します。IdPについては学認技術ガイドの記述内容などを参考に設定を行なってください。

SPの設定は「[3.3 学内システムとして構築する場合の設定](#%E5%AD%A6%E5%86%85%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0%E3%81%A8%E3%81%97%E3%81%A6%E6%A7%8B%E7%AF%89%E3%81%99%E3%82%8B%E5%A0%B4%E5%90%88%E3%81%AE%E8%A8%AD%E5%AE%9A)」にて行います。

> 設定手順は学認技術ガイドに記されている最低限のものとなります。個別の状況に対応するより詳細な設定が必要となる場合は構築後に`shibboleth2.xml`などの設定ファイルを編集してください。

## 準備

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

### 前提条件の確認

このNotebookを実行するための前提条件を満たしていることを確認します。

前提となる条件を以下に示します。

* 構築した際のパラメータが group_vars ファイルに記録されていること
* Ansibleから操作可能であること
* Ansibleから管理者権限でコマンドを実行できること
* Moodleコンテナが実行中であること
* Moodleコンテナでphp-fpmが実行されていること

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

対象となるホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

Moodleコンテナが実行中であることを確認します。

In [ ]:
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    docker compose ps --status running moodle | grep moodle'

Moodleコンテナでphp-fpmが実行されていることを確認します。

In [ ]:
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    docker compose exec moodle supervisorctl status || \
    docker compose exec moodle ls /run/php-fpm/www.sock'

## パラメータ設定

### MoodleのURL

MoodleではサイトのURLを`config.php`の`$CFG->wwwroot`に設定しています。HTTPS接続にすることによりURLが変更される場合はその値を指定します。

> 公開するURLが現在の設定値から変更がない場合はこの節をスキップして次節に進んでください。

現在設定されている URL の値を確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_url"])

新たなURLの値を次のセルで指定してください。

In [ ]:
# (例)
# moodle_url = 'https://moodle.example.org'

moodle_url = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

値の保存を行う前に入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
check_parameters(
    'moodle_url',
    nb_vars=locals(),
)

パラメータの値を group_vars ファイルに保存します。

In [ ]:
from urllib.parse import urlparse
%run scripts/group.py

res = urlparse(moodle_url)
update_group_vars(
    target_group,
    _file="30-httpd.yml",
    moodle_url=moodle_url,
    server_name=res.netloc,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-httpd.yml

### 学認フェデレーションに関するパラメータ

学認技術ガイドの「[shibboleth2.xml　ファイル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158266)」に説明されているパラメータの設定を行います。

> この節は**学認フェデレーションに参加する場合のみ**指定が必要となります。学認フェデレーションに参加せずに学内システムとして構築するなどの場合はこの節をスキップして次節に進んでください。

この節では以下のパラメータを指定します。

* DSサーバのURL
* メタデータのURL

参加するフェデレーションによって指定する値は異なります。以下のリンク先などを先などを参照して適切な値を指定してください。

* [shibboleth2.xml ファイル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158266)
* [テストフェデレーションのルール](https://www.gakunin.jp/join/test/rule)

#### DSサーバ

DSサーバのURLを次のセルで指定してください。

In [ ]:
# (例)
# url_gakunin_ds = 'https://ds.gakunin.nii.ac.jp/WAYF'          # 運用フェデレーション
# url_gakunin_ds = 'https://test-ds.gakunin.nii.ac.jp/WAYF' # テストフェデレーション

url_gakunin_ds = 

#### メタデータ

学認フェデレーションのメタデータを取得するURLを次のセルで指定してください。

In [ ]:
# (例)
# url_gakunin_metadata = 'https://metadata.gakunin.nii.ac.jp/gakunin-metadata.xml'          # 運用フェデレーション
# url_gakunin_metadata = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-metadata.xml'  # テストフェデレーション

url_gakunin_metadata = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="30-shibboleth.yml",
    url_gakunin_ds=url_gakunin_ds,
    url_gakunin_metadata=url_gakunin_metadata,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-shibboleth.yml

### 学内システムとして構築する場合の設定

学認技術ガイドの「[学内システムとして構築する場合の設定](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158282)」の記述に従い、特定のIdP・特定のSPの間でのみ連携する設定を行います。このNotebookでは Shibboleth SP 側の設定のみ扱います。IdP側の設定は技術ガイドの記述などを参考に設定を行ってください。

> 学認フェデレーションに参加している場合はこの節をスキップして先に進んでください。

Shibboleth SPに対して設定する以下の項目の値を指定します。

* IdP の entityID
* IdP のメタデータファイル

#### IdPのentity ID

SPと連携するIdPの entity ID を次のセルで指定してください。

In [ ]:
# (例)
# idp_entity_id = 'https://idp.example.org/idp/shibboleth' 

idp_entity_id = 

#### IdPのメタデータ

連携するIdPのメタデータのファイルを次のセルで指定してください。

指定するメタデータファイルは事前にこのNotebook環境にアップロードしておいてください。次のセルにはNotebook環境でのパスを指定してください。

In [ ]:
# (例)
# idp_metadata= 'idp-metadata.xml' 

idp_metadata= 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    target_group,
    _file="30-shibboleth.yml",
    idp_entity_id=idp_entity_id,
    idp_metadata=str(Path(idp_metadata).resolve()),
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-shibboleth.yml

## メンテナンスモードへの切り替え

設定を行う際にMoodleコンテナを一時停止することになるので、メンテナンスモードに切り替えます。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

## 証明書の配置

Shibbolethコンテナが使用する証明書、秘密鍵を配置します。

Shibbolethコンテナが使用する証明書、秘密鍵を以下に示します。

1. Apache HTTP serverが mod_ssl で利用するサーバ証明書
1. Shibboleth SPが署名、暗号化のために利用するサーバ証明書
1. 学認フェデレーションのメタデータを検証するための証明書（学認フェデレーションを利用する場合のみ）

通常の運用では1と2の証明書、秘密鍵は同じものとなります。そのため、ここでは1と2を個別に指定せずに１組の証明書、秘密鍵のみを指定します。

### 証明書と秘密鍵の指定

Apache HTTP serverとShibboleth SPで利用する証明書と秘密鍵を配置します。

> ここで指定するサーバ証明書と秘密鍵は事前にこのNotebook環境にアップロードしておいてください。

ここで指定した証明書、秘密鍵は構築環境に配置されShibbolethコンテナにbindマウントされます。配置先のパスとコンテナにおけるパスの関係を次表に示します。

|構築環境の配置先となるパス|コンテナでのパス|説明|
|:---|:---|:---|
|/srv/moodle/shibboleth/conf/httpd/certs/server.crt|/etc/pki/tls/certs/server.crt|Apache HTTP serverのサーバ証明書|
|/srv/moodle/shibboleth/conf/httpd/certs/server.key|/etc/pki/tls/private/server.key|Apache HTTP serverの秘密鍵|

bindマウントされた証明書、秘密鍵のファイルはApache HTTP serverから利用されます。さらにコンテナの起動時に、これらのファイルはShibboleth SPで利用するファイルとしてコピーされます。Shibboleth SPが利用する証明書、秘密鍵のパスを次表に示します。

|コンテナ内でのパス|説明|
|:---|:---|
|/etc/shibboleth/cert/server.crt|Shibboleth SPのサーバ証明書|
|/etc/shibboleth/cert/server.key|Shibboleth SPの秘密鍵|

Shibboleth SPが利用するサーバ証明書、秘密鍵をApache HTTP server が利用するものと別に用意するのは、それぞれの用途に応じたファイルの所有者、グループの設定が必要となるためです。

またShibbolethコンテナによる証明書のコピーは上記の表に示したパスに既にファイルが存在している場合は実行されません。SP用の証明書、秘密鍵をコンテナにbindマウントするように設定するするとコンテナ起動時のコピーを抑制することができます。これによりApache HTTP serverとShibboleth SPとで異なる証明書を利用するなどの例外的な状況に対応してすることが可能となります。

 ####  サーバ証明書の指定
 
 サーバ証明書のパスを次のセルで指定してください。ここで指定するパスはNotebook環境におけるサーバ証明書のパスになります。

 証明書のフォーマットはPEMであることを想定しています。 またサーバ証明書に中間CA証明書を連結する場合は[電子証明書発行サービス-サーバ動作事例集](https://certs.nii.ac.jp/manual/manuals/reports/03)で説明されているように、「サーバ証明書」、「中間CA証明書」の順に記載してください。

In [ ]:
# (例)
# moodle_cert_path = './certs/server.crt'

moodle_cert_path = 

#### 秘密鍵の指定

秘密鍵のパスを次のセルで指定してください。ここで指定するパスはNotebook環境における秘密鍵のパスになります。

In [ ]:
# (例)
# moodle_private_key_path = './certs/server.key'

moodle_private_key_path = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    target_group,
    _file="30-httpd.yml",
    moodle_cert_path=str(Path(moodle_cert_path).resolve()),
    moodle_private_key_path=str(Path(moodle_private_key_path).resolve()),
    httpd="shibboleth",
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-httpd.yml

### メタデータを検証するための証明書の指定

学認フェデレーションのメタデータを検証するための証明書を指定します。　

> この節は**学認フェデレーションに参加する場合のみ**指定が必要となります。学認フェデレーションに参加せずに学内システムとして構築するなどの場合はこの節をスキップして次節に進んでください。

次のセルで学認のメタデータの署名を検証するための署名証明書を指定してください。

ここで指定する証明書は参加するフェデレーションによって異なります。以下に示すリンク先の情報などを参考にしてください。

* [メタデータ署名証明書 -- 運用フェデレーション用](https://meatwiki.nii.ac.jp/confluence/display/GakuNinShibInstall/signer)
* [テストフェデレーションの概要とルール](https://www.gakunin.jp/join/test/rule)
* [shibboleth2.xml ファイル](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158266)

In [ ]:
# (例)
# gakunin_signer = 'https://metadata.gakunin.nii.ac.jp/gakunin-signer-2017.cer'           # 運用フェデレーション
# gakunin_signer = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-signer-2020.cer'  # テストフェデレーション
# gakunin_signer = './certs/gakunin-signer.cer'　　# Notebook環境のパスで指定する場合

gakunin_signer = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
from urllib.parse import urlparse
from pathlib import Path

url_gakunin_signer = urlparse(gakunin_signer)
if len(url_gakunin_signer.scheme) > 0:
    remove_group_vars(target_group, 'path_gakunin_signer')
    update_group_vars(
        target_group,
        _file="30-shibboleth.yml",
        url_gakunin_signer=gakunin_signer,
    )
else:
    remove_group_vars(target_group, 'url_gakunin_signer')
    update_group_vars(
        target_group,
        _file="30-shibboleth.yml",
        path_gakunin_signer=str(Path(gakunin_signer).resolve()),
    )

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-shibboleth.yml

### 証明書と秘密鍵を構築環境に配置する

次のセルを実行すると証明書、秘密鍵を構築環境に配置する ansible の playbook を実行します。

実際の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-shibboleth-certs.yml || true

実際に証明書の配置を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-shibboleth-certs.yml

サーバ証明書がMoodle環境に配置されたことを確認するために、証明書の内容を表示してみます。

In [ ]:
!ansible {target_group} -a \
    'openssl x509 -in /srv/moodle/shibboleth/conf/httpd/certs/server.crt \
    -noout -text'

配置した秘密鍵を表示します。

In [ ]:
!ansible {target_group} -b -a \
    'openssl rsa -in /srv/moodle/shibboleth/conf/httpd/certs/server.key \
    -noout -text'

## 設定ファイルの配置

Shibbolethコンテナで実行する Apache HTTP server と Shibboleth SP の設定ファイルを構築環境のホスト側に配置します。

bind mountによって配置する設定ファイルのホスト環境におけるパスとコンテナ環境におけるパスの対応関係を以下に示します。

|コンテナ名|コンテナ環境のパス|ホスト環境のパス|
|:---|:---|:---|
|shibboleth|/etc/httpd/conf/httpd.conf|/srv/moodle/shibboleth/conf/httpd/conf/httpd.conf|
|shibboleth|/etc/httpd/conf.d/|/srv/moodle/shibboleth/conf/httpd/conf.d/|
|shibboleth|/etc/httpd/conf.modules.d/|/srv/moodle/shibboleth/conf/httpd/conf.modules.d/|
|shibboleth|/etc/shibboleth/|/srv/moodle/shibboleth/conf/shibboleth/|

実際の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-shibboleth-conf.yml

実際に設定ファイルの配置を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-shibboleth-conf.yml

設定ファイルがコピーされたことを確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    tree shibboleth/conf'

## コンテナ構成の変更

MoodleとShibboleth SPが連携できるようにするようコンテナ構成を変更します。

### docker-compose.yml の配置

このNotebookの設定手順ではHTTPS接続に対応するためのコンテナを追加する変更を行います。新たなコンテナ構成に対応する`docker-compose.yml`を構築環境に配置します。ここで配置する`docker-compose.yml`は元の設定に対して以下に示す変更をおこなっています。

* Shibbolethコンテナを追加する
* moodleコンテナで実行している `php-fpm` のソケットをShibbolethコンテナを共有するように bind マウントを設定する
* moodleコンテナのhttpdサービスを無効化する
* moodleコンテナの`config.php`で設定している `$CFG->wwwroot` の値を変更する

実際に`docker-compose.yml`を配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

`docker-compose.yml`を構築環境に配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

### コンテナ設定の変更を反映する

コンテナ設定の変更反映をするためにコンテナを再起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

`State`が`Up`となっていない場合はコンテナのログなどを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose logs --tail=50 shibboleth'

## メンテナンスモードの解除

設定変更が完了したのでメンテナンスモードを解除します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

Moodle環境にアクセスして利用可能であることを確認します。

次のセルを実行するとMoodleのアドレスを表示します。表示されたリンクをクリックしてMoodleにアクセスできることを確認してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_url"])

## MoodleのShibbolethプラグインの設定

MoodleのShibboleth認証を利用できるようにする設定を行います。

### Shibboleth認証を有効にする

次のセルを実行すると表示されるリンクからMoodleの認証管理画面を開いてください。
> メニューから[サイト管理]--[プラグイン]--[認証]--[認証管理]を選択して同様の画面を開くことができます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/admin/settings.php?section=manageauths" target="_blank">{1}</a>'.format(
    load_group_var(target_group, 'moodle_url'), u'認証管理'))

以下のような画面が表示されるので Shibboleth の Yes の列をクリックし Shibboleth認証プラグインを有効にしてください。

![認証管理](images/moodle-221-03.png)

### Shibbolethプラグイン設定

Shibbolethプラグインの設定を行います。

次のセルを実行すると表示されるリンクからShibbolethプラグインの設定画面を開いてください。
> メニューから[サイト管理]--[プラグイン]--[認証]--[Shibboleth]を選択して同様の画面を開くことができます。

In [ ]:
HTML(u'<a href="{0}/admin/settings.php?section=authsettingshibboleth" target="_blank">{1}</a>'.format(
    load_group_var(target_group, 'moodle_url'), u'Shibboleth 設定'))

以下のような画面が表示されるので Shibboleth認証に関する設定を行ってください。

![Shibboleth設定](images/moodle-221-04.png)

Shibbolethコンテナでは学認により[テンプレート](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=12158175)として公開されている `attribute-map.xml`を利用しています。`attribute-map.xml`で設定されている属性名をMoodleのデータマッピングの設定に指定してください。

例えば以下のように指定を行います。

|項目名|設定例|
|---|---|
|ユーザ名|eppn|
|データマッピング：名|jaGivenName|
|データマッピング：姓|jasn|
|データマッピング：メールアドレス|mail|
|データマッピング：所属組織|jao|
|データマッピング：部署|jaou|
|データマッピング：名 - ヨミガナ|givenName|
|データマッピング：姓 - ヨミガナ|sn|
